# Get GFS data from the source
- Data dictionary: https://rda.ucar.edu/datasets/ds084.1/#metadata/grib2.html?_do=y
- Data access: https://rda.ucar.edu/datasets/ds084.1/#!access
- https://github.com/ecmwf/cfgrib/issues/138
- https://github.com/ecmwf/cfgrib/issues/75

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime, timedelta
from pathlib import Path
from typing import Iterable

import xarray as xr
import zarr

In [ ]:
path = Path("../data/")
for file in sorted(path.glob("*.grib2")):
    if file.stem.split("f00")[-1] != "0":
        print(file)
        break

## Load into xarray

In [ ]:
ds_t = xr.open_dataset(
    file,
    engine="cfgrib",
    backend_kwargs={
        "filter_by_keys": {
            "cfVarName": "t",
            "typeOfLevel": "surface",
            "stepType": "instant",
        }
    },
)
ds_t = ds_t.t

In [ ]:
ds_dp = xr.open_dataset(
    file,
    engine="cfgrib",
    backend_kwargs={"filter_by_keys": {"typeOfLevel": "surface", "stepType": "avg"}},
)
da_d = ds_dp.dlwrf
da_p = ds_dp.prate

In [ ]:
ds_v = xr.open_dataset(
    file,
    engine="cfgrib",
    backend_kwargs={
        "filter_by_keys": {"cfVarName": "v", "typeOfLevel": "isobaricInhPa"},
        "indexpath": "",
    },
)
da_v = ds_v.isel(isobaricInhPa=0).v

In [ ]:
ds_u = xr.open_dataset(
    file,
    engine="cfgrib",
    backend_kwargs={
        "filter_by_keys": {"cfVarName": "u", "typeOfLevel": "isobaricInhPa"},
        "indexpath": "",
    },
)
ds_u = ds_u.isel(isobaricInhPa=0).u

In [ ]:
ds_merged = xr.merge([ds_t, ds_u, ds_v, ds_d, ds_p])

In [ ]:
ds_merged.to_zarr(f"../zarrs/{file.stem}")